In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.special import wrightomega, entr
from quant_inf.tools.projection_matrix import Projection
from quant_inf.tools import coeff_to_matrix,random_coefficients_gaussian
import time
from itertools import product

In [2]:
sns.set_context("notebook", font_scale=1)

# Chambolle-Pock algorithm

In [4]:
def chambolle_pock(F,projection,tau=3.,sig=.3,tol:float=1e-12,max_iter:int=1000):
    n = projection.n
    k = len(F)
    F_completed = np.zeros((n,n))
    F_completed[:k,:k] = F
    F_completed = projection(F_completed)
    x = np.eye(n)
    y = np.zeros((n,n))
    x_bar = x
    logs_dual = []
    logs_primal = []

    for i in range(max_iter):
        # prox step on dual variable
        arg_prox = y + sig*x_bar
        proj = projection(arg_prox)
        y = arg_prox - proj - (sig - (np.trace(proj)/n))*np.eye(n)

        # prox step on primal variable
        arg_prox = x - tau*y
        eigval,eigvect = np.linalg.eigh(n*(F_completed + arg_prox/tau))
        x_new = (eigvect * (tau/n) * np.real(wrightomega(eigval - np.log(tau/n)))) @ eigvect.T

        x_bar = 2*x_new - x
        x = x_new

        eig = np.linalg.eigvalsh(n*(F_completed-y))
        dual_value = np.trace(y) + np.sum(np.exp(eig))/n - 1
        logs_dual.append(dual_value)

        #Computing feasible x
        x_feasible = projection(x)
        x_feasible = x_feasible + (1 - (np.trace(x_feasible))/n)*np.eye(n)
        eig = np.linalg.eigvalsh(x_feasible)
        min_eig = np.min(eig)
        if min_eig<0:
            u=(-min_eig)/(1-min_eig)
        else:
            u=0
        x_feasible = (1-u)*x_feasible + u*np.eye(n)
        eig = (1-u)*eig + u*np.ones(n)
        eig = eig*(eig>0) #some value may remains negative up to machine precision
        primal_value = np.sum(F*x_feasible[:k,:k]) - (1/n)*np.sum(-entr(eig))
        logs_primal.append(primal_value)

        if np.abs(dual_value-primal_value) < tol:
                break

    return np.array(logs_primal),np.array(logs_dual)

# Decrease of duality gap along iterations

### For $\varphi(x) = (1,x)$

In [5]:
results = pd.DataFrame()

number_sample = 5
tol = 1e-10

d = 50

tol = 1e-10
features = (
    [set()] 
    + [{i} for i in range(1,d+1)]
)

complete_graph_features = (
    [{i} for i in range(1,d+1)]
    + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
)

projection = Projection(features)

for k in range(number_sample):
    F = coeff_to_matrix(random_coefficients_gaussian(complete_graph_features),d)

    logs_primal, logs_dual= chambolle_pock(F,projection,tol=tol,tau=3,sig=.3)

    results = pd.concat(
        [results,
         pd.DataFrame({"d": d,
                       "iteration": range(len(logs_primal)),
                       "gap":logs_dual-logs_primal,
                       "n_sample": k,
                       "param": "sig = .3, tau =3"
                       }
        )
        ]
    )

for k in range(number_sample):
    F = coeff_to_matrix(random_coefficients_gaussian(complete_graph_features),d)

    logs_primal,logs_dual= chambolle_pock(F,projection,tol=tol, tau=.9,sig=.9)

    results = pd.concat(
        [results,
         pd.DataFrame({"d": d,
                       "iteration": range(len(logs_primal)),
                       "gap":logs_dual-logs_primal,
                       "n_sample": k+number_sample,
                       "param": "sig = tau = .9"
                       }
        )
        ]
    )

In [ ]:
sns.lineplot(
    data=results,
    x="iteration",
    y="gap",
    units="n_sample",
    estimator=None,
    hue = "param"
)
plt.yscale("log")
plt.ylim(top = 100,bottom=tol)
plt.ylabel("duality gap")
plt.show()

### Higher order hierarchy

In [7]:
results = pd.DataFrame()

number_sample = 5
tol = 1e-10
max_iter = 5000
d = 10
n = 50

tol = 1e-10
features = (
    [set()] 
    + [{i} for i in range(1,d+1)]
)

complete_graph_features = (
    [{i} for i in range(1,d+1)]
    + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
)

all_features = (
    [set()] 
    + [{i} for i in range(1,d+1)]
    + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
    + [{i,j,k} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1)]
    + [{i,j,k,l} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1) for l in range(k+1,d+1)]
    + [{i,j,k,l,m} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1) for l in range(k+1,d+1) for m in range(l+1,d+1)]
)

projection = Projection(all_features[:n])

for k in range(number_sample):
    F = coeff_to_matrix(random_coefficients_gaussian(complete_graph_features),d)

    logs_primal,logs_dual= chambolle_pock(F,projection,tol=tol,tau=3,sig=.3,max_iter=max_iter)

    results = pd.concat(
        [results,
         pd.DataFrame({"d": d,
                       "iteration": range(len(logs_primal)),
                       "gap":logs_dual-logs_primal,
                       "n_sample": k,
                       "param": "sig = .3, tau =3"
                       }
        )
        ]
    )

for k in range(number_sample):
    F = coeff_to_matrix(random_coefficients_gaussian(complete_graph_features),d)

    logs_primal,logs_dual= chambolle_pock(F,projection,tol=tol,tau=.9,sig=.9,max_iter=max_iter)

    results = pd.concat(
        [results,
         pd.DataFrame({"d": d,
                       "iteration": range(len(logs_primal)),
                       "gap":logs_dual-logs_primal,
                       "n_sample": k,
                       "param": "sig = tau = .9"
                       }
        )
        ]
    )


In [ ]:
sns.lineplot(
    data=results,
    x="iteration",
    y="gap",
    units="n_sample",
    estimator=None,
    hue = "param"
)
plt.yscale("log")
plt.ylim(top = 100,bottom=np.min(results["gap"]))
plt.ylabel("duality gap")
plt.show()

# Number of iterations v.s. d

In [ ]:
results = pd.DataFrame(columns=['d','tol','n_iter','run_time','n_sample'])

number_sample = 5

liste_d =range(10,101,10)
list_tol = [1e-8]

for tol,d,k in product(list_tol,liste_d,range(number_sample)):
    features = (
        [set()] 
        + [{i} for i in range(1,d+1)]
    )

    complete_graph_features = (
        [{i} for i in range(1,d+1)]
        + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
    )

    projection = Projection(features)

    F = coeff_to_matrix(random_coefficients_gaussian(complete_graph_features),d)

    start = time.perf_counter()
    logs_primal,logs_dual= chambolle_pock(F,projection,tol=1e-6)
    run_time = time.perf_counter() - start

    results.loc[len(results)] = [
        d,
        tol,
        len(logs_primal),
        run_time,
        k
        ] 


In [ ]:
results["n_sample"] = results["n_sample"].astype("category")
sns.scatterplot(
    data=results,
    x="d",
    y="n_iter",
    legend=None
)
plt.show()

# Number of iterations v.s. n

In [11]:
results = pd.DataFrame(columns=['n','tol','n_iter','run_time','n_sample'])

number_sample = 5

d = 10
liste_n =range(11,101,10)
tol = 1e-2
max_iter=10000

complete_graph_features = (
    [{i} for i in range(1,d+1)]
    + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
)

all_features = (
    [set()] 
    + [{i} for i in range(1,d+1)]
    + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
    + [{i,j,k} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1)]
    + [{i,j,k,l} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1) for l in range(k+1,d+1)]
    + [{i,j,k,l,m} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1) for l in range(k+1,d+1) for m in range(l+1,d+1)]
)

for n_sample in range(number_sample):
    
    F = coeff_to_matrix(random_coefficients_gaussian(complete_graph_features),d)
    
    for n in liste_n:
        features = all_features[:n]
        projection = Projection(features)

        start = time.perf_counter()
        logs_primal,logs_dual= chambolle_pock(F,projection,tol=tol,max_iter=max_iter)
        run_time = time.perf_counter() - start

        results.loc[len(results)] = [
            n,
            tol,
            len(logs_primal),
            run_time,
            n_sample+1
            ] 


In [ ]:
results["n_sample"] = results["n_sample"].astype("category")
sns.scatterplot(
    data=results,
    x="n",
    y="n_iter",
    hue="n_sample",
    legend=None
)
plt.show()